# Global Sensitivity Analysis

#### Imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mesa.batchrunner import BatchRunner
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
from itertools import combinations
from src.model import SchellingModel

The parameters that are varied are: `density`, `p_random`, `min_tenure`, `u_threshold`, and `alpha`. The model reporters are the last values of the `dissimilarity` and `exposure` metric. 

We use 128 Sobol samples and 5 replicates.

In [ ]:
parameters = {
    'num_vars': 5,
    'names': ['density', 'p_random', 'min_tenure', 'u_threshold', 'alpha'],
    'bounds': [[0.7, 0.95], [0.05, 0.2], [3, 6], [7, 9.5], [0.1, 0.7]]
}
model_steps = 200

model_reporters = {"dissimilarity": lambda m: np.mean(m.dissimilarity_list[-1]),
                   "exposure": lambda m: np.mean(m.exposure_list[-1])}

replicates = 5
distinct_samples = 128
param_values = saltelli.sample(parameters, distinct_samples, calc_second_order=True)

Now, we calculate the first, second, and total order Sobol indices

In [ ]:
batch = BatchRunner(SchellingModel, 
                    max_steps=model_steps,
                    variable_parameters={name:[] for name in parameters['names']},
                    model_reporters=model_reporters)

# Seed the process for reproducibility
n_runs = replicates * len(param_values)
rng = np.random.RandomState(1234)
seeds = rng.randint(0, 2**31 - 1, size=n_runs)


count = 0
data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['density', 'p_random', 'min_tenure', 'u_threshold', 'alpha'])
data['Run'], data['dissimilarity'], data['exposure'] = None, None, None

for i in range(replicates):
    for vals in param_values: 
        vals = list(vals)
        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(parameters['names'], vals):
            variable_parameters[name] = val
        variable_parameters['seedje'] = int(seeds[count])

        batch.run_iteration(variable_parameters, tuple(vals) + (seeds[count],), count)
        iteration_data = batch.get_model_vars_dataframe().iloc[count]
        iteration_data['Run'] = count
        data.iloc[count, 0:5] = vals
        data.iloc[count, 5:8] = iteration_data
        count += 1

        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')

data.to_csv('SA_data.csv', index=False)

The data calculated above is saved into `SA_data.csv`. The run used for the report is stored in `SA_data_best_run.csv`.

In [ ]:
SA_data = pd.read_csv('SA_data_best_run.csv')
# SA_data = pd.read_csv('SA_data.csv')

Si_dissimilarity = sobol.analyze(parameters, SA_data['dissimilarity'].values, print_to_console=False)
Si_exposure = sobol.analyze(parameters, SA_data['exposure'].values, print_to_console=False)

We calculate the sum of the first and second order to observe their impact.

In [ ]:
# 1) Sum of first‐order indices
first_sum = np.sum(Si_dissimilarity['S1'])
print(f"Sum of first‐order Sobol indices: {first_sum:.4f}")

# 2) Sum of second‐order indices
p = len(parameters['names'])
flat_s2 = Si_dissimilarity['S2'].reshape(p*p)      
flat_s2 = flat_s2[~np.isnan(flat_s2)]
second_sum = np.sum(flat_s2)
print(f"Sum of second‐order Sobol indices: {second_sum:.4f}")

# 3) Combined up to second order
print(f"Combined up to 2nd order:   {first_sum + second_sum:.4f}")

Finally, we plot the results:

In [ ]:
def plot_index(s, params, i, idx, title=''):
    """
    Creates a plot for Sobol sensitivity analysis that shows the contributions
    of each parameter to the global sensitivity.

    Args:
        s (dict): dictionary {'S#': dict, 'S#_conf': dict} of dicts that hold
            the values for a set of parameters
        params (list): the parameters taken from s
        i (str): string that indicates what order the sensitivity is.
        title (str): title for the plot
    """

    if i == '2':
        p = len(single_labels)
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']

    l = len(indices)

    # plt.figure(figsize=(5,3))
    plt.subplot(2,2,idx)
    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params, fontsize=13)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o', color='#a26da4', capsize=5)
    plt.axvline(0, c='k')
    plt.xticks(fontsize=12)
    if i == 'T':
        plt.yticks([])
    
    plt.grid(ls='dashed')


pretty_names = {
    'density': r'$\rho$',
    'p_random': r'$p_r$',
    'min_tenure': r'$\bar{t}$',
    'u_threshold': r'$\bar{u}$',
    'alpha': r'$\alpha$'
}
single_labels = [pretty_names[n] for n in parameters['names']]
pair_labels   = [
    f"({pretty_names[a]}, {pretty_names[b]})"
    for a, b in combinations(parameters['names'], 2)
]

for Si, metric in zip((Si_dissimilarity, Si_exposure), ('GSA of Dissimilarity', 'GSA of Exposure')):
    plt.figure(figsize=(6,6), dpi=300)
    plt.suptitle(f'{metric}', fontsize=14, y=0.95)
    plot_index(Si, single_labels, '1', 1, f'First Order Sensitivity')
    plot_index(Si, pair_labels, '2', 3, f'Second Order Sensitivity')
    plot_index(Si, single_labels, 'T', 2, f'Total Order Sensitivity')
    plt.subplots_adjust(hspace=0.25, wspace=0.15)

    plt.show()
